In [1]:
import telebot
from telebot import types
from ultralytics import YOLO
from PIL import Image
import cv2
import os
import sys
import shutil
import numpy as np
import array
from pydub import AudioSegment
import soundfile as sf


from transformers import pipeline

from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd

/home/semenov-denis/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-21 23:42:00 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [4]:
class TestBot:
    def __init__(self, token=None):
        self.bot = telebot.TeleBot('6505268192:AAFk3XrQjkDHvSziBe2Wq5OLLqAxLnIF7bg')
        self.pipe = pipeline("automatic-speech-recognition", model="openai/whisper-medium")
        self.model = YOLO('yolov8x-seg.pt')

    #audio generation
        models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
            "facebook/fastspeech2-en-ljspeech",
            arg_overrides={"vocoder": "hifigan", "fp16": False}
        )
        
        TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
        generator = task.build_generator(models, cfg)
        self.audio_addons = [models, cfg, task, generator]
        
        self.audio_model = TTSHubInterface
        
        

    def start_test(self, message):
        self.bot.send_message(message.chat.id, "Привет! Скинь фото, а я скажу, что на нем расположено. Но учти, я распознаю пока мало объектов")

    def get_text_messages(self, message):
        self.bot.send_message(message.chat.id, "Ну я же сказал скинуть фото")

    def handle_voise(self, message):
        file_info = self.bot.get_file(message.voice.file_id)
        downloaded_file = self.bot.download_file(file_info.file_path)
        with open('new_file.ogg', 'wb') as new_file:
            new_file.write(downloaded_file)
        prediction = self.pipe('new_file.ogg', batch_size=8)["text"]
        self.bot.send_message(message.chat.id, prediction)

        # text to audio
        sample = self.audio_model.get_model_input(self.audio_addons[2], prediction)
        wav, rate = self.audio_model.get_prediction(self.audio_addons[2], self.audio_addons[0][0], self.audio_addons[3], sample)
        sf.write('new_file.wav', wav, rate, 'PCM_24')

        #AudioSegment.from_wav("new_file.wav").export("new_file.ogg", format="ogg")
        
        audio = open(r'new_file.wav', 'rb')
        self.bot.send_audio(message.chat.id, audio)
        audio.close()
        
        
        
        
   
    def handle_photo(self, message):
        print('message.photo =', message.photo)
        fileID = message.photo[-1].file_id
        print('fileID =', fileID)
        file_info = self.bot.get_file(fileID)
        print('file.file_path =', file_info.file_path)
        downloaded_file = self.bot.download_file(file_info.file_path)
        with open("image.jpg", 'wb') as new_file:
            new_file.write(downloaded_file)
        try:
            shutil.rmtree('/home/semenov-denis/bots/output/')
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))
        results = self.model.predict(source="image.jpg", save=True, project="/home/semenov-denis/bots/output/") # Display preds. Accepts all YOLO predict arguments
        with open('/home/semenov-denis/bots/output/predict/image.jpg', 'rb') as ret:
            self.bot.send_photo(message.chat.id, ret)
        

       

#im1 = Image.open("bus.jpg")
#results = model.predict(source=im1, save=True)  # save plotted images

# from ndarray
#im2 = cv2.imread("bus.jpg")
#results = model.predict(source=im2, save=True, save_txt=True)  # save predictions as labels

# from list of PIL/ndarray
#results = model.predict(source=[im1, im2])
            
    
        

    

    

# Replace 'YOUR_TELEGRAM_BOT_TOKEN' with your actual bot token.
#bot_token = 'YOUR_TELEGRAM_BOT_TOKEN'
test_bot = TestBot() #bot_token)

@test_bot.bot.message_handler(commands=['start'])
def handle_start(message):
    test_bot.start_test(message)

@test_bot.bot.message_handler(content_types=['text'])
def handle_message(message):
    test_bot.get_text_messages(message)

@test_bot.bot.message_handler(content_types=['photo'])
def handle_photo(message):
    test_bot.handle_photo(message)

@test_bot.bot.message_handler(content_types=['voice'])
def handle_voise(message):
    test_bot.handle_voise(message)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Fetching 9 files: 100%|████████████████████████| 9/9 [00:00<00:00, 63980.91it/s]
2023-10-21 23:43:06 | INFO | fairseq.tasks.speech_to_text | dictionary size (vocab.txt): 75
/home/semenov-denis/.local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
2023-10-21 23:43:06 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from /home/semenov-denis/.cache/fairseq/models--facebook--fastspeech2-en-ljspeech/snapshots/a3e3e5e2e62bb7ca7514b11aa469e9c5b01a20bf/hifigan.bin


In [5]:
test_bot.bot.polling()

ReadTimeout: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=25)

In [ ]:
model = models[0]

In [ ]:

ipd.Audio(wav, rate=rate)

In [ ]:
task

In [ ]:
rate

In [ ]:
sample = test_bot.audio_model.get_model_input(test_bot.audio_addons[2], "check, check")
wav, rate = test_bot.audio_model.get_prediction(test_bot.audio_addons[2], test_bot.audio_addons[0][0], test_bot.audio_addons[3], sample)

In [ ]:
f = open('new_file.ogg', 'rb')
f.read()